In [ ]:
import os
from PIL import Image
import cv2, sys, re
import pandas as pd
import numpy as np
import random
from keras import models,Sequential,layers
from keras.models import Model, Input
from keras.layers import Conv2D, SeparableConv2D, Dense, MaxPooling2D, GlobalAveragePooling2D, AveragePooling2D
from keras.layers import Activation, BatchNormalization, Dropout, Flatten, Reshape
from keras.layers import Add
from keras.utils import to_categorical
from keras.callbacks import Callback
from keras.optimizers import SGD,Adam
from keras.regularizers import l2
from keras.datasets import cifar10
import numpy as np
import keras.backend as K
from sklearn.metrics import log_loss
import glob
from keras.models import load_model
import tensorflow as tf
from sklearn.ensemble import RandomForestClassifier

import tensorflow as tf

from keras import backend as K
from keras.models import Model

from keras.layers import Conv2D, BatchNormalization, ReLU, DepthwiseConv2D, Activation, Input, Add
from keras.layers import GlobalAveragePooling2D, Reshape, Dense, multiply, Softmax, Flatten

# ** to update custom Activate functions
from keras.utils.generic_utils import get_custom_objects

In [ ]:
len(all_train_dirs)

In [ ]:
all_train_dirs = glob.glob('/kaggle/input/all-data/dfdc_train_part/' + 'dfdc_train_part_*')
# for i, train_dir in enumerate(all_train_dirs):
#     print('[{:02}]'.format(i), train_dir)
all_train_dirs.sort()
del all_train_dirs[12]

In [ ]:
all_train_dirs = glob.glob('/kaggle/input/deepfake-detection-faces-part-*/')

In [ ]:
len(all_train_dirs[0])

In [ ]:
print(all_train_dirs[12])

In [ ]:
def read_image(path,file_list,label_img):
    a = [0,50,100,150,200,250,300]
    im = []
    label = []
    for i in file_list:
        path_dir = path +'/' + i
        for j in a:
            try:
                img = Image.open(path_dir + '/' + str(j) +'.png')
                arr_img = list(np.array(img))
                im.append(arr_img)

                label_list = np.array(label_img.iloc[:, [3]])

                if i + '.mp4' in label_list:
                    label.append(1)  # Deepfake
                else:
                    label.append(0)
            except:
                pass
    return im, label

In [ ]:
eye_cascade = cv2.CascadeClassifier('/kaggle/input/haar-cascades-for-face-detection/haarcascade_eye.xml')

def detection_eyes(a):
    roi_gray = a[0:160,0:160]
    eyes = eye_cascade.detectMultiScale(roi_gray)
    cnt_eyes = 0
    for (ex,ey,ew,eh) in eyes:
        cnt_eyes += 1
    return cnt_eyes

def read_image(path,file_list,label_img):
    a = [150,250]
    im = []
    label = []
    for i in file_list:
        path_dir = path +'/' + i
        for j in a:
            try:
                img = Image.open(path_dir + '/' + str(j) +'.png')
                arr_img = np.array(img)
                if detection_eyes(arr_img) == 1:
#                 if detection_eyes(arr_img) != 1:
                    im.append(list(arr_img))
                    label_list = np.array(label_img.iloc[:, [2]])
                    if i + '.mp4' in label_list:
                        label.append(1)  # Deepfake
                    else:
                        label.append(0)
            except:
                pass
    return im, label

In [ ]:
# 전체 데이터셋 real:deep = 21184:1199
# 폴더 11개 데이터셋 real:deep = 6851:504 (결과 잘 나왔었음)

X = []
y = []
# len(all_train_dirs) range 안에 대입
for i in range(1):
    path = all_train_dirs[i]
    file_list = os.listdir(path)
    label = pd.read_csv(all_train_dirs[i] + '/metadata.csv',delimiter=',')
    img,label = read_image(path,file_list,label)
    X += img
    y += label

X = np.array(X)
y = np.array(y).reshape(-1,1)

In [ ]:
len(X)

In [ ]:
import matplotlib.pyplot as plt


eye_cascade = cv2.CascadeClassifier('/kaggle/input/haar-cascades-for-face-detection/haarcascade_eye.xml')
def detection_eyes(a):
#     img = cv2.rectangle(a,(0,0),(160,160),(255,0,0),2)
#     gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    roi_gray = a[0:160,0:160]
#     roi_color = img[0:160,0:160]
    eyes = eye_cascade.detectMultiScale(roi_gray)
    cnt_eyes = 0
    for (ex,ey,ew,eh) in eyes:
        cnt_eyes += 1
#         cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)
    return cnt_eyes

A = []
A_side = []
y = np.array([])
y_side = np.array([])

for k,a in enumerate(X):
    if detection_eyes(a) == 1:
        A.append(X[k])
        y = np.append(y,B[k])
    else:
        A_side.append(X[k])
        y_side = np.append(y,B[k])
A = np.squeeze(np.array(X))
A_side = np.squeeze(np.array(X_side))


In [ ]:
# 데이터셋 0,1로 분류해서 인덱싱 하기 위함
c = []
d = []
e = []
f = []


for i in range(len(y)):
    if y[i] == 0:
        c.append(y[i])
        d.append(X[i])

for i in range(len(y)):
    if y[i] == 1:
        e.append(y[i])
        f.append(X[i])

X = np.array(f+d)
y = np.array(e+c).reshape(-1,1)
print(len(X))
print(len(y))

In [ ]:
a=0
for i in range(len(y)):
    if y[i] == 0:
        a += 1
print(a)

In [ ]:
del c
del d
del e
del f

In [ ]:
A = X[2000:14000]
B = y[2000:14000]

In [ ]:
del X
del y

In [ ]:
from keras.layers import GlobalAveragePooling2D, GlobalMaxPooling2D, Reshape, Dense, multiply, Permute, Concatenate, Conv2D, Add, Activation, Lambda
from keras import backend as K
from keras.activations import sigmoid


def cbam_block(cbam_feature, ratio=8):
	cbam_feature = channel_attention(cbam_feature, ratio)
	cbam_feature = spatial_attention(cbam_feature)
	return cbam_feature

def channel_attention(input_feature, ratio=8):
	
	channel_axis = 1 if K.image_data_format() == "channels_first" else -1
	channel = input_feature._keras_shape[channel_axis]
	
	shared_layer_one = Dense(channel//ratio,
							 activation='relu',
							 kernel_initializer='he_normal',
							 use_bias=True,
							 bias_initializer='zeros')
	shared_layer_two = Dense(channel,
							 kernel_initializer='he_normal',
							 use_bias=True,
							 bias_initializer='zeros')
	
	avg_pool = GlobalAveragePooling2D()(input_feature)    
	avg_pool = Reshape((1,1,channel))(avg_pool)
	assert avg_pool._keras_shape[1:] == (1,1,channel)
	avg_pool = shared_layer_one(avg_pool)
	assert avg_pool._keras_shape[1:] == (1,1,channel//ratio)
	avg_pool = shared_layer_two(avg_pool)
	assert avg_pool._keras_shape[1:] == (1,1,channel)
	
	max_pool = GlobalMaxPooling2D()(input_feature)
	max_pool = Reshape((1,1,channel))(max_pool)
	assert max_pool._keras_shape[1:] == (1,1,channel)
	max_pool = shared_layer_one(max_pool)
	assert max_pool._keras_shape[1:] == (1,1,channel//ratio)
	max_pool = shared_layer_two(max_pool)
	assert max_pool._keras_shape[1:] == (1,1,channel)
	
	cbam_feature = Add()([avg_pool,max_pool])
	cbam_feature = Activation('sigmoid')(cbam_feature)
	
	if K.image_data_format() == "channels_first":
		cbam_feature = Permute((3, 1, 2))(cbam_feature)
	
	return multiply([input_feature, cbam_feature])

def spatial_attention(input_feature):
	kernel_size = 1024
	
	if K.image_data_format() == "channels_first":
		channel = input_feature._keras_shape[1]
		cbam_feature = Permute((2,3,1))(input_feature)
	else:
		channel = input_feature._keras_shape[-1]
		cbam_feature = input_feature
	
	avg_pool = Lambda(lambda x: K.mean(x, axis=3, keepdims=True))(cbam_feature)
	assert avg_pool._keras_shape[-1] == 1
	max_pool = Lambda(lambda x: K.max(x, axis=3, keepdims=True))(cbam_feature)
	assert max_pool._keras_shape[-1] == 1
	concat = Concatenate(axis=3)([avg_pool, max_pool])
	assert concat._keras_shape[-1] == 2
	cbam_feature = Conv2D(filters = 1,
					kernel_size=kernel_size,
					strides=1,
					padding='same',
					activation='sigmoid',
					kernel_initializer='he_normal',
					use_bias=False)(concat)	
	assert cbam_feature._keras_shape[-1] == 1
	
	if K.image_data_format() == "channels_first":
		cbam_feature = Permute((3, 1, 2))(cbam_feature)
		
	return multiply([input_feature, cbam_feature])

In [ ]:
def divide(x, y, train_prop):
    random.seed(1234)
    tmp = np.random.permutation(np.arange(len(x)))
    X_train = x[tmp][:-(len(x)-round(train_prop * len(x)))]
    y_train = y[tmp][:-(len(x)-round(train_prop * len(x)))]
    X_test = x[tmp][-(len(x)-round(train_prop * len(x))):]
    y_test = y[tmp][-(len(x)-round(train_prop * len(x))):]
    return X_train, y_train, X_test, y_test

X_train, y_train, X_test, y_test = divide(X,y,0.7)

In [ ]:
del A
del B

In [ ]:
def divide(x, y):
    random.seed(1234)
    tmp = np.random.permutation(np.arange(len(x)))
    X_train = x[tmp][:]
    y_train = y[tmp][:]
    return X_train, y_train

X_train, y_train = divide(A,B)

In [ ]:
def conv2d_bn(x, filters, kernel_size, padding='same', strides=1, activation='relu', weight_decay=1e-5):
    x = Conv2D(filters, kernel_size, padding=padding, strides=strides, kernel_regularizer=l2(weight_decay))(x)
    x = BatchNormalization()(x)
    if activation:
        x = Activation(activation)(x)

    return x

def sepconv2d_bn(x, filters, kernel_size, padding='same', strides=1, activation='relu', weight_decay=1e-5,
                 depth_multiplier=1):
    x = SeparableConv2D(filters, kernel_size, padding=padding, strides=strides, depth_multiplier=depth_multiplier,
                        depthwise_regularizer=l2(weight_decay), pointwise_regularizer=l2(weight_decay))(x)
    x = BatchNormalization()(x)

    if activation:
        x = Activation(activation)(x)

    return x


def Xception(model_input, classes):
    ## Entry flow
    x = conv2d_bn(model_input, 32, (3, 3), strides=2)  # (299, 299, 3) -> (150, 150, 32)
    x = conv2d_bn(x, 64, (3, 3))

    for fliters in [128, 256, 728]:  # (75, 75, 64) -> (75, 75, 128) -> (38, 38, 256) -> (19, 19, 728)
        residual = conv2d_bn(x, fliters, (1, 1), strides=2, activation=None)

        x = Activation(activation='relu')(x)
        x = sepconv2d_bn(x, fliters, (3, 3))
        x = sepconv2d_bn(x, fliters, (3, 3), activation=None)
        x = MaxPooling2D((3, 3), padding='same', strides=2)(x)
        # 1X1 convolution layer를 activation 대신에 마지막에 적용시킨다.
        x = Add()([x, residual])

    ## Middle flow
    for i in range(8):  # (19, 19, 728)
        residual = x

        x = sepconv2d_bn(x, 728, (3, 3))
        x = sepconv2d_bn(x, 728, (3, 3))
        x = sepconv2d_bn(x, 728, (3, 3), activation=None)

        x = Add()([x, residual])

    ## Exit flow
    residual = conv2d_bn(x, 1024, (1, 1), strides=2, activation=None)  # (19, 19, 728) -> (10, 10, 1024)

    x = Activation(activation='relu')(x)
    x = sepconv2d_bn(x, 728, (3, 3))
    x = sepconv2d_bn(x, 1024, (3, 3), activation=None)  # (19, 19, 728) -> (19, 19, 1024)
    x = MaxPooling2D((3, 3), padding='same', strides=2)(x)  # (19, 19, 1024) -> (10, 10, 1024)

    x = Add()([x, residual])

    x = sepconv2d_bn(x, 1536, (3, 3))
    x = sepconv2d_bn(x, 2048, (3, 3))
    x = cbam_block(x)
    
    x = GlobalAveragePooling2D()(x)

    ## Optinal fully-connected layers
    
#     x = Dense(4096)(x)
#     x = BatchNormalization()(x)
#     x = Activation(activation='relu')(x)

#     x = Dense(4096)(x)
#     x = BatchNormalization()(x)
#     x = Activation(activation='relu')(x)
    
    x = Dropout(0.5)(x)
#     x = BatchNormalization()(x)

    model_output = Dense(classes, activation='softmax')(x)
    model = Model(model_input, model_output, name='Xception')
    return model

class LearningRateSchedule(Callback):
    def on_epoch_end(self, epoch, logs=None):
        if (epoch+1)%2 == 0:
            lr = K.get_value(self.model.optimizer.lr)
            K.set_value(self.model.optimizer.lr, lr*0.94)

input_shape = (160, 160, 3)

X_train = X_train.astype('float32')/255.
# X_test = X_test.astype('float32')/255.

y_train = to_categorical(y_train, num_classes=2)
# y_test = to_categorical(y_test, num_classes=2)

model_input = Input(shape=input_shape)

model = Xception(model_input,2)

optimizer = SGD(lr=0.0009, momentum=0.9)
# optimizer = Adam(lr=0.0017, beta_1=0.9, beta_2=0.999, epsilon=10e-8, decay=1e-7, amsgrad=False)

callbacks_list = [LearningRateSchedule()]

model.compile(optimizer, loss='categorical_crossentropy', metrics=['acc'])

model.fit(X_train, y_train, batch_size=64, epochs=15, validation_split=0.25, callbacks=callbacks_list)

# y_pred = model.predict(X_test)
             
# a = log_loss(y_test,y_pred)
# print(a)

In [ ]:
# mobile net

import tensorflow as tf

from keras import backend as K
from keras.models import Model

from keras.layers import Conv2D, BatchNormalization, ReLU, DepthwiseConv2D, Activation, Input, Add
from keras.layers import GlobalAveragePooling2D, Reshape, Dense, multiply, Softmax, Flatten

# ** to update custom Activate functions
from keras.utils.generic_utils import get_custom_objects


""" Define layers block functions """
def Hswish(x):
    return x * tf.nn.relu6(x + 3) / 6

# ** update custom Activate functions
get_custom_objects().update({'custom_activation': Activation(Hswish)})


def __conv2d_block(_inputs, filters, kernel, strides, is_use_bias=False, padding='same', activation='RE', name=None):
    x = Conv2D(filters, kernel, strides= strides, padding=padding, use_bias=is_use_bias)(_inputs)
    x = BatchNormalization()(x)
    if activation == 'RE':
        x = ReLU(name=name)(x)
    elif activation == 'HS':
        x = Activation(Hswish, name=name)(x)
    else:
        raise NotImplementedError
    return x

def __depthwise_block(_inputs, kernel=(3, 3), strides=(1, 1), activation='RE', is_use_se=True, num_layers=0):
    x = DepthwiseConv2D(kernel_size=kernel, strides=strides, depth_multiplier=1, padding='same')(_inputs)
    x = BatchNormalization()(x)
    if is_use_se:
        x = __se_block(x)
    if activation == 'RE':
        x = ReLU()(x)
    elif activation == 'HS':
        x = Activation(Hswish)(x)
    else:
        raise NotImplementedError
    return x

def __global_depthwise_block(_inputs):
    assert _inputs._keras_shape[1] == _inputs._keras_shape[2]
    kernel_size = _inputs._keras_shape[1]
    x = DepthwiseConv2D((kernel_size, kernel_size), strides=(1, 1), depth_multiplier=1, padding='valid')(_inputs)
    return x

def __se_block(_inputs, ratio=4, pooling_type='avg'):
    filters = _inputs._keras_shape[-1]
    se_shape = (1, 1, filters)
    if pooling_type == 'avg':
        se = GlobalAveragePooling2D()(_inputs)
    elif pooling_type == 'depthwise':
        se = __global_depthwise_block(_inputs)
    else:
        raise NotImplementedError
    se = Reshape(se_shape)(se)
    se = Dense(filters // ratio, activation='relu', kernel_initializer='he_normal', use_bias=False)(se)
    se = Dense(filters, activation='hard_sigmoid', kernel_initializer='he_normal', use_bias=False)(se)
    return multiply([_inputs, se])

def __bottleneck_block(_inputs, out_dim, kernel, strides, expansion_dim, is_use_bias=False, shortcut=True, is_use_se=True, activation='RE', num_layers=0, *args):
    with tf.name_scope('bottleneck_block'):
        # ** to high dim 
        bottleneck_dim = expansion_dim

        # ** pointwise conv 
        x = __conv2d_block(_inputs, bottleneck_dim, kernel=(1, 1), strides=(1, 1), is_use_bias=is_use_bias, activation=activation)

        # ** depthwise conv
        x = __depthwise_block(x, kernel=kernel, strides=strides, is_use_se=is_use_se, activation=activation, num_layers=num_layers)

        # ** pointwise conv
        x = Conv2D(out_dim, (1, 1), strides=(1, 1), padding='same')(x)
        x = BatchNormalization()(x)

        if shortcut and strides == (1, 1):
            in_dim = K.int_shape(_inputs)[-1]
            if in_dim != out_dim:
                ins = Conv2D(out_dim, (1, 1), strides=(1, 1), padding='same')(_inputs)
                x = Add()([x, ins])
            else:
                x = Add()([x, _inputs])
    return x

def build_mobilenet_v3(input_size, num_classes, model_type='large', pooling_type='avg', include_top=True):
    # ** input layer
    inputs = Input(shape=(input_size, input_size, 3))

    # ** feature extraction layers
    net = __conv2d_block(inputs, 16, kernel=(3, 3), strides=(2, 2), is_use_bias=False, padding='same', activation='HS') 
    if model_type == 'large':
        config_list = large_config_list
    elif model_type == 'small':
        config_list = small_config_list
    else:
        raise NotImplementedError
        
    for config in config_list:
        net = __bottleneck_block(net, *config)
    
    # ** final layers
    net = __conv2d_block(net, 960, kernel=(3, 3), strides=(1, 1), is_use_bias=True, padding='same', activation='HS', name='output_map')
    net = cbam_block(net)
    net = AveragePooling2D((3,3), padding='same', strides=2)(net)
    print(net.shape)
#     if pooling_type == 'avg':
#         net = GlobalAveragePooling2D()(net)

#     elif pooling_type == 'depthwise':
#         net = __global_depthwise_block(net)
#     else:
#         raise NotImplementedError

    net = BatchNormalization()(net)

    a = Flatten()(net)
    a = Dense(8640)(a)
    a = BatchNormalization()(a)
    a = Activation(activation='relu')(a)
    
#     a = Dense(8640)(a)
#     a = BatchNormalization()(a)
#     a = Activation(activation='relu')(a)

#     a = Dropout(0.5)(a)
    a = BatchNormalization()(a)

    
    model_output = Dense(num_classes, activation='softmax')(a)
    model = Model(inputs=inputs, outputs=model_output)

    return model

""" define bottleneck structure """
# ** 
# **             
global large_config_list    
global small_config_list

large_config_list = [[16,  (3, 3), (1, 1), 16,  False, False, False, 'RE',  0],
                     [24,  (3, 3), (2, 2), 64,  False, False, False, 'RE',  1],
                     [24,  (3, 3), (1, 1), 72,  False, True,  False, 'RE',  2],
                     [40,  (5, 5), (2, 2), 72,  False, False, True,  'RE',  3],
                     [40,  (5, 5), (1, 1), 120, False, True,  True,  'RE',  4],
                     [40,  (5, 5), (1, 1), 120, False, True,  True,  'RE',  5],
                     [80,  (3, 3), (2, 2), 240, False, False, False, 'RE',  6],
                     [80,  (3, 3), (1, 1), 200, False, True,  False, 'RE',  7],
                     [80,  (3, 3), (1, 1), 184, False, True,  False, 'RE',  8],
                     [80,  (3, 3), (1, 1), 184, False, True,  False, 'RE',  9],
                     [112, (3, 3), (1, 1), 480, False, False, True,  'RE', 10],
                     [112, (3, 3), (1, 1), 672, False, True,  True,  'RE', 11],
                     [160, (5, 5), (1, 1), 672, False, False, True,  'RE', 12],
                     [160, (5, 5), (2, 2), 672, False, True,  True,  'RE', 13],
                     [160, (5, 5), (1, 1), 960, False, True,  True,  'RE', 14]]

small_config_list = [[16,  (3, 3), (2, 2), 16,  False, False, True,  'RE', 0],
                     [24,  (3, 3), (2, 2), 72,  False, False, False, 'RE', 1],
                     [24,  (3, 3), (1, 1), 88,  False, True,  False, 'RE', 2],
                     [40,  (5, 5), (1, 1), 96,  False, False, True,  'RE', 3],
                     [40,  (5, 5), (1, 1), 240, False, True,  True,  'RE', 4], 
                     [40,  (5, 5), (1, 1), 240, False, True,  True,  'RE', 5],
                     [48,  (5, 5), (1, 1), 120, False, False, True,  'RE', 6],
                     [48,  (5, 5), (1, 1), 144, False, True,  True,  'RE', 7],
                     [96,  (5, 5), (2, 2), 288, False, False, True,  'RE', 8],
                     [96,  (5, 5), (1, 1), 576, False, True,  True,  'RE', 9],
                     [96,  (5, 5), (1, 1), 576, False, True,  True,  'RE', 10]]

# large_config_list = [[16,  (3, 3), (1, 1), 16,  False, False, False, 'RE',  0],
#                      [24,  (3, 3), (2, 2), 64,  False, False, False, 'RE',  1],
#                      [24,  (3, 3), (1, 1), 72,  False, True,  False, 'RE',  2],
#                      [40,  (5, 5), (2, 2), 72,  False, False, True,  'RE',  3],
#                      [40,  (5, 5), (1, 1), 120, False, True,  True,  'RE',  4],
#                      [40,  (5, 5), (1, 1), 120, False, True,  True,  'RE',  5],
#                      [80,  (3, 3), (2, 2), 240, False, False, False, 'HS',  6],
#                      [80,  (3, 3), (1, 1), 200, False, True,  False, 'HS',  7],
#                      [80,  (3, 3), (1, 1), 184, False, True,  False, 'HS',  8],
#                      [80,  (3, 3), (1, 1), 184, False, True,  False, 'HS',  9],
#                      [112, (3, 3), (1, 1), 480, False, False, True,  'HS', 10],
#                      [112, (3, 3), (1, 1), 672, False, True,  True,  'HS', 11],
#                      [160, (5, 5), (1, 1), 672, False, False, True,  'HS', 12],
#                      [160, (5, 5), (2, 2), 672, False, True,  True,  'HS', 13],
#                      [160, (5, 5), (1, 1), 960, False, True,  True,  'HS', 14]]

# small_config_list = [[16,  (3, 3), (2, 2), 16,  False, False, True,  'RE', 0],
#                      [24,  (3, 3), (2, 2), 72,  False, False, False, 'RE', 1],
#                      [24,  (3, 3), (1, 1), 88,  False, True,  False, 'RE', 2],
#                      [40,  (5, 5), (1, 1), 96,  False, False, True,  'HS', 3],
#                      [40,  (5, 5), (1, 1), 240, False, True,  True,  'HS', 4], 
#                      [40,  (5, 5), (1, 1), 240, False, True,  True,  'HS', 5],
#                      [48,  (5, 5), (1, 1), 120, False, False, True,  'HS', 6],
#                      [48,  (5, 5), (1, 1), 144, False, True,  True,  'HS', 7],
#                      [96,  (5, 5), (2, 2), 288, False, False, True,  'HS', 8],
#                      [96,  (5, 5), (1, 1), 576, False, True,  True,  'HS', 9],
#                      [96,  (5, 5), (1, 1), 576, False, True,  True,  'HS', 10]]


""" build MobileNet V3 model """

def learning_rate_scheduler(epoch):
    if epoch > 50:
        lr = 1e-3
    elif epoch > 100:
        lr = 1e-4
    elif epoch > 300:
        lr = 5e-5
    elif epoch > 500:
        lr = 1e-5
    elif epoch > 800:
        lr = 1e-6
    else:
        lr = 3e-2
    return lr

class LearningRateSchedule(Callback):
    def on_epoch_end(self, epoch, logs=None):
        if (epoch+1)%2 == 0:
            lr = K.get_value(self.model.optimizer.lr)
            K.set_value(self.model.optimizer.lr, lr*0.94)


# X_train = X_train.astype('float32')/255.
# X_test = X_test.astype('float32')/255.

# y_train = to_categorical(y_train, num_classes=2)
# y_test = to_categorical(y_test, num_classes=2)

model = build_mobilenet_v3(input_size=160, num_classes=2, model_type='large', pooling_type='avg', include_top=True)

optimizer = SGD(lr=0.05, momentum=0.9)
# optimizer = Adam(lr=1e-2, beta_1=0.9, beta_2=0.999, epsilon=10e-8, decay=1e-3, amsgrad=False)

callbacks_list = [LearningRateSchedule()]

model.compile(optimizer, loss='categorical_crossentropy', metrics=['acc'])

model.fit(X_train, y_train, batch_size=128, epochs=300, validation_split=0.25, callbacks=callbacks_list)

y_pred = model.predict(X_test)
             
a = log_loss(y_test,y_pred)
print(a)

In [ ]:
y_pred[0]

In [ ]:
a = y_pred[:,1]
b = y_test[:,1]

In [ ]:
print(y_pred[:2])

In [ ]:
print(type(y_pred[0]))

In [ ]:
print(y_test[5:10])
print(y_pred[5:10])


In [ ]:
log_loss(y_test[5:10],y_pred[5:10])

In [ ]:
print(y_test[390:400])

In [ ]:
print(y_pred[:,0])

In [ ]:
model.save('mobilenet2.h5')

In [ ]:
a = pd.read_csv('/kaggle/input/deepfake-detection-faces-part-7-2/metadata.csv')
for i in range(600):
    print(a.iloc[i,:])

In [ ]:
model.summary()

In [ ]:
# Feature map 이미지 비교 위한 코딩

from keras.models import load_model
import tensorflow as tf
import matplotlib.pyplot as plt

# Layer가 많을수록 다양한 feature map을 얻는 것을 확인했다.

def show_image(img_path, target_size):
    img = tf.keras.preprocessing.image.load_img(img_path, target_size=target_size)
    img_tensor = tf.keras.preprocessing.image.img_to_array(img)

    # print(img_tensor[0, 0])                         # [31.  2.  6.]
    # print(np.min(img_tensor), np.max(img_tensor))   # 0.0 255.0
    # print(img_tensor.dtype, img_tensor.shape)       # float32 (150, 150, 3)

    # 스케일링하지 않으면 이미지 출력 안됨
    img_tensor /= 255

    plt.imshow(img_tensor)
    plt.show()

show_image('/kaggle/input/deepfake-detection-faces-part-3-1/qawbflilul/150.png', target_size=(150, 150))

In [ ]:
# keras의 load_model을 이용하면 에러가 발생

# model = load_model('/kaggle/input/image-model/image_model_0.2955.h5')
model = tf.keras.models.load_model('/kaggle/input/image-model/image_model_0.2955.h5')


# model = tf.keras.models.load_model('/kaggle/input/mobilenet/mobilenet.h5')

# model.summary()

In [ ]:
model.summary()

In [ ]:
# feature map 시각화

# 옆모습의 특징을 정면보다 잘 뽑아내지 못한다


# 이미지 불러오기
def load_image(img_path, target_size):
    img = tf.keras.preprocessing.image.load_img(img_path, target_size=target_size)
    img_tensor = tf.keras.preprocessing.image.img_to_array(img)

    # 배치 사이즈 추가 + 스케일링 결과 반환
    return img_tensor[np.newaxis] / 255     # (1, 150, 150, 3)


# 첫 번째 등장하는 컨볼루션 레이어의 모든 피처맵(32개) 출력
def show_first_feature_map(loaded_model, img_path):
    first_output = loaded_model.layers[2].output
    print(first_output.shape, first_output.dtype)   # (?, 148, 148, 32) <dtype: 'float32'>

    # 1개의 출력을 갖는 새로운 모델 생성
    model = tf.keras.models.Model(inputs=loaded_model.input, outputs=first_output)

    # 입력으로부터 높이와 너비를 사용해서 target_size에 해당하는 튜플 생성
    target_size = (loaded_model.input.shape[2], loaded_model.input.shape[2])
    img_tensor = load_image(img_path, target_size)

    print(loaded_model.input.shape)     # (?, 150, 150, 3)
    print(img_tensor.shape)             # (1, 150, 150, 3)

    first_activation = model.predict(img_tensor)

    # 컨볼루션 레이어에서 필터 크기(3), 스트라이드(1), 패딩(valid)을 사용했기 때문에
    # 150에서 148로 크기가 일부 줄었음을 알 수 있다. 필터 개수는 32.
    print(first_activation.shape)       # (1, 148, 148, 32)
    print(first_activation[0, 0, 0])    # [0.00675746 0. 0.02397328 0.03818807 0. ...]

    # 19번째 활성 맵 출력. 기본 cmap은 viridis. gray는 흑백 컬러맵.
    # [0, :, :, feature_index]
    # 0은 첫 번째 데이터(원본 이미지)의 피처맵을 가리킨다. 사진은 1장만 사용했기 때문에 0만 가능
    # 가운데 콜론(:)은 높이와 너비를 가리키는 차원의 모든 데이터
    # feature_index는 보고 싶은 피처맵이 있는 채널을 가리킨다.
    # 32개의 필터를 사용했다면 0부터 31까지의 피처맵이 존재한다.
    plt.figure(figsize=(16, 8))
    for i in range(first_activation.shape[-1]):
        plt.subplot(4, 8, i + 1)

        # 눈금 제거. fignum은 같은 피켜에 연속 출력
        plt.axis('off')
        plt.matshow(first_activation[0, :, :, i], cmap='gray', fignum=0)
    plt.tight_layout()
    plt.show()

# chhcmabekn = fake, 
show_first_feature_map(model,'/kaggle/input/deepfake-detection-faces-part-3-0/vezsoxophr/150.png')

In [ ]:

# model = tf.keras.Sequential()

# model.add(Conv2D(32, kernel_size=(3, 3), input_shape=(160, 160, 1), activation='relu'))
# model.add(Conv2D(64, (3, 3), activation='relu'))
# model.add(Conv2D(128, (3, 3), activation='relu'))
# model.add(MaxPooling2D(pool_size=2))
# model.add(MaxPooling2D(pool_size=2))

In [ ]:
print(y[:10])

In [ ]:
print(y_pred)

In [ ]:
for i in range(len(y_pred)):
    if y_pred[i][0] > 0.09:
        print(y_pred[i])

In [ ]:
print(y_test)

In [ ]:
from keras.models import load_model

In [ ]:
from keras.models import Model
from keras import layers
from keras.layers import Dense, Input, BatchNormalization, Activation
from keras.layers import Conv2D, SeparableConv2D, MaxPooling2D, GlobalAveragePooling2D, GlobalMaxPooling2D
from keras_applications.imagenet_utils import _obtain_input_shape
# from keras.utils.data_utils import get_file

# WEIGHTS_PATH = 'https://github.com/fchollet/deep-learning-models/releases/download/v0.4/xception_weights_tf_dim_ordering_tf_kernels.h5'

def Xception():

	# Determine proper input shape
	input_shape = _obtain_input_shape(None, default_size=160, min_size=71, data_format='channels_last',require_flatten=False)

	img_input = Input(shape=input_shape)

	# Block 1
	x = Conv2D(32, (3, 3), strides=(2, 2), use_bias=False)(img_input)
	x = BatchNormalization()(x)
	x = Activation('relu')(x)
	x = Conv2D(64, (3, 3), use_bias=False)(x)
	x = BatchNormalization()(x)
	x = Activation('relu')(x)

	residual = Conv2D(128, (1, 1), strides=(2, 2), padding='same', use_bias=False)(x)
	residual = BatchNormalization()(residual)

	# Block 2
	x = SeparableConv2D(128, (3, 3), padding='same', use_bias=False)(x)
	x = BatchNormalization()(x)
	x = Activation('relu')(x)
	x = SeparableConv2D(128, (3, 3), padding='same', use_bias=False)(x)
	x = BatchNormalization()(x)

	# Block 2 Pool
	x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
	x = layers.add([x, residual])

	residual = Conv2D(256, (1, 1), strides=(2, 2), padding='same', use_bias=False)(x)
	residual = BatchNormalization()(residual)

	# Block 3
	x = Activation('relu')(x)
	x = SeparableConv2D(256, (3, 3), padding='same', use_bias=False)(x)
	x = BatchNormalization()(x)
	x = Activation('relu')(x)
	x = SeparableConv2D(256, (3, 3), padding='same', use_bias=False)(x)
	x = BatchNormalization()(x)

	# Block 3 Pool
	x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
	x = layers.add([x, residual])

	residual = Conv2D(728, (1, 1), strides=(2, 2), padding='same', use_bias=False)(x)
	residual = BatchNormalization()(residual)

	# Block 4
	x = Activation('relu')(x)
	x = SeparableConv2D(728, (3, 3), padding='same', use_bias=False)(x)
	x = BatchNormalization()(x)
	x = Activation('relu')(x)
	x = SeparableConv2D(728, (3, 3), padding='same', use_bias=False)(x)
	x = BatchNormalization()(x)

	x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
	x = layers.add([x, residual])

	# Block 5 - 12
	for i in range(8):
		residual = x

		x = Activation('relu')(x)
		x = SeparableConv2D(728, (3, 3), padding='same', use_bias=False)(x)
		x = BatchNormalization()(x)
		x = Activation('relu')(x)
		x = SeparableConv2D(728, (3, 3), padding='same', use_bias=False)(x)
		x = BatchNormalization()(x)
		x = Activation('relu')(x)
		x = SeparableConv2D(728, (3, 3), padding='same', use_bias=False)(x)
		x = BatchNormalization()(x)

		x = layers.add([x, residual])

	residual = Conv2D(1024, (1, 1), strides=(2, 2), padding='same', use_bias=False)(x)
	residual = BatchNormalization()(residual)

	# Block 13
	x = Activation('relu')(x)
	x = SeparableConv2D(728, (3, 3), padding='same', use_bias=False)(x)
	x = BatchNormalization()(x)
	x = Activation('relu')(x)
	x = SeparableConv2D(1024, (3, 3), padding='same', use_bias=False)(x)
	x = BatchNormalization()(x)

	# Block 13 Pool
	x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
	x = layers.add([x, residual])

	# Block 14
	x = SeparableConv2D(1536, (3, 3), padding='same', use_bias=False)(x)
	x = BatchNormalization()(x)
	x = Activation('relu')(x)

	# Block 14 part 2
	x = SeparableConv2D(2048, (3, 3), padding='same', use_bias=False)(x)
	x = BatchNormalization()(x)
	x = Activation('relu')(x)

	# Fully Connected Layer
	x = GlobalAveragePooling2D()(x)
	x = Dense(1, activation='sigmoid')(x)

	inputs = img_input

	# Create model
	model = Model(inputs, x, name='xception')

	# Download and cache the Xception weights file
# 	weights_path = get_file('xception_weights.h5', WEIGHTS_PATH, cache_subdir='models')

	# load weights
# 	model.load_weights(weights_path)

	return model


"""
	Instantiate the model by using the following line of code
	model = Xception()
"""


# input_shape = (160, 160, 3)

X_train = X_train.astype('float32')/255.
X_test = X_test.astype('float32')/255.

# y_train = to_categorical(y_train, num_classes=2)
# y_test = to_categorical(y_test, num_classes=2)

# model_input = Input(shape=input_shape)

model = Xception()

# optimizer = SGD(lr=0.0009, momentum=0.9)
optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=10e-8, decay=1e-5, amsgrad=False)
class LearningRateSchedule(Callback):
    def on_epoch_end(self, epoch, logs=None):
        if (epoch+1)%2 == 0:
            lr = K.get_value(self.model.optimizer.lr)
            K.set_value(self.model.optimizer.lr, lr*0.94)
callbacks_list = [LearningRateSchedule()]

model.compile(optimizer, loss='binary_crossentropy', metrics=['acc'])

model.fit(X_train, y_train, batch_size=64, epochs=40, validation_split=0.25, callbacks=callbacks_list)

In [ ]:
def conv2d_bn(x, filters, kernel_size, padding='same', strides=1, activation='relu', weight_decay=1e-5):
    x = Conv2D(filters, kernel_size, padding=padding, strides=strides, kernel_regularizer=l2(weight_decay))(x)
    x = BatchNormalization()(x)
    if activation:
        x = Activation(activation)(x)
    return x

def sepconv2d_bn(x, filters, kernel_size, padding='same', strides=1, activation='relu', weight_decay=1e-5,
                 depth_multiplier=1):
    x = SeparableConv2D(filters, kernel_size, padding=padding, strides=strides, depth_multiplier=depth_multiplier,
                        depthwise_regularizer=l2(weight_decay), pointwise_regularizer=l2(weight_decay))(x)
    x = BatchNormalization()(x)

    if activation:
        x = Activation(activation)(x)
    return x


def Xception(model_input, classes):
    ## Entry flow
    x = conv2d_bn(model_input, 8, (3, 3), strides=2)  # (299, 299, 3) -> (150, 150, 32)
    x = conv2d_bn(x, 16, (3, 3))

    for fliters in [32, 64, 128]:  # (75, 75, 64) -> (75, 75, 128) -> (38, 38, 256) -> (19, 19, 728)
        residual = conv2d_bn(x, fliters, (1, 1), strides=2, activation=None)

        x = Activation(activation='relu')(x)
        x = sepconv2d_bn(x, fliters, (3, 3))
        x = sepconv2d_bn(x, fliters, (3, 3), activation=None)
        x = MaxPooling2D((3, 3), padding='same', strides=2)(x)
        # 1X1 convolution layer를 activation 대신에 마지막에 적용시킨다.
        x = Add()([x, residual])

    ## Middle flow
    for i in range(8):  # (19, 19, 728)
        residual = x

        x = sepconv2d_bn(x, 128, (3, 3))
        x = sepconv2d_bn(x, 128, (3, 3))
        x = sepconv2d_bn(x, 128, (3, 3), activation=None)

        x = Add()([x, residual])

    ## Exit flow
    residual = conv2d_bn(x, 256, (1, 1), strides=2, activation=None)  # (19, 19, 728) -> (10, 10, 1024)

    x = Activation(activation='relu')(x)
    x = sepconv2d_bn(x, 192, (3, 3))
    x = sepconv2d_bn(x, 256, (3, 3), activation=None)  # (19, 19, 728) -> (19, 19, 1024)
    x = MaxPooling2D((3, 3), padding='same', strides=2)(x)  # (19, 19, 1024) -> (10, 10, 1024)

    x = Add()([x, residual])

    x = sepconv2d_bn(x, 384, (3, 3))
    x = sepconv2d_bn(x, 512, (3, 3))
    x = cbam_block(x)
    a = Flatten()(x)
    a = Dense(12800)(a)
    a = BatchNormalization()(a)
    a = Activation(activation='relu')(a) # (1,12800)
    a = Dropout(0.5)(a)
    
    model_output = Dense(classes, activation='softmax')(a)
    model = Model(model_input, model_output, name='Xception')
    return model

class LearningRateSchedule(Callback):
    def on_epoch_end(self, epoch, logs=None):
        if (epoch+1)%2 == 0:
            lr = K.get_value(self.model.optimizer.lr)
            K.set_value(self.model.optimizer.lr, lr*0.94)

input_shape = (160, 160, 3)

X_train = X_train.astype('float32')/255.
X_test = X_test.astype('float32')/255.

y_train = to_categorical(y_train, num_classes=2)
y_test = to_categorical(y_test, num_classes=2)

model_input = Input(shape=input_shape)

model = Xception(model_input,2)

# optimizer = SGD(lr=0.0009, momentum=0.9)
optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=10e-8, decay=1e-5, amsgrad=False)

callbacks_list = [LearningRateSchedule()]

model.compile(optimizer, loss='categorical_crossentropy', metrics=['acc'])

model.fit(X_train, y_train, batch_size=64, epochs=40, validation_split=0.25, callbacks=callbacks_list)

y_pred = model.predict(X_test)

a = log_loss(y_test,y_pred)
print(a)

In [ ]:
model.save('XceptionDense.h5')

In [ ]:
print(y_test[10:20])

In [ ]:
print(y_pred[10:20])

In [ ]:
for i in range(len(X_train)):
    a = model.predict(X_train[i].reshape(1,160,160,3))
    print(a)

In [ ]:
def divide(x, y, train_prop):
    random.seed(1234)
    tmp = np.random.permutation(np.arange(len(x)))
    X_train = x[tmp][:-(len(x)-round(train_prop * len(x)))]
    y_train = y[tmp][:-(len(x)-round(train_prop * len(x)))]
    X_test = x[tmp][-(len(x)-round(train_prop * len(x))):]
    y_test = y[tmp][-(len(x)-round(train_prop * len(x))):]
    return X_train, y_train, X_test, y_test

X_train, y_train, X_test, y_test = divide(A,B,0.7)

In [ ]:
model.save('image_model_test1.h5')

In [ ]:
X = []
y = []

for i in range(11,20):
    path = all_train_dirs[i]
    file_list = os.listdir(path)
    label = pd.read_csv(all_train_dirs[i] + '/metadata.csv',delimiter=',')
    img,label = read_image(path,file_list,label)
    X += img
    y += label

X = np.array(X)
y = np.array(y).reshape(-1,1)

In [ ]:
X_train = X_train.astype('float32')/255.
X_test = X_test.astype('float32')/255.

y_pred = model.predict(X_test)
a = log_loss(y_test,y_pred)
print(a)

In [ ]:
model = tf.keras.models.load_model('/kaggle/input/imagemodel/image_model_0.3030.h5')

In [ ]:
model.summary()

In [ ]:
model.save('image_test.h5')

In [ ]:
import os
from PIL import Image
import cv2, sys, re
import pandas as pd
import numpy as np
import random
from keras import models,Sequential,layers
from keras.models import Model, Input
from keras.layers import Conv2D, SeparableConv2D, Dense, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Activation, BatchNormalization, Dropout,Flatten,Reshape
from keras.layers import Add
from keras.utils import to_categorical
from keras.callbacks import Callback
from keras.optimizers import SGD,Adam
from keras.regularizers import l2
from keras.datasets import cifar10
import numpy as np
import keras.backend as K
from sklearn.metrics import log_loss
import glob
from keras.models import load_model
import tensorflow as tf
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import keras

In [ ]:
all_train_dirs = glob.glob('/kaggle/input/' + 'deepfake-detection-faces-*')

model = tf.keras.models.load_model('/kaggle/input/imagemodel/image_model_0.3030.h5')
# model = tf.keras.models.load_model('/kaggle/input/image-test1/image_test.h5')


def read_image(path,file_list,label_img):
    a = [1]
    im = []
    label = []
    for i in file_list:
        path_dir = path +'/' + i
        for j in a:
            try:
                img = Image.open(path_dir + '/' + str(j) +'.png')
#                 img = img[0,:,:,0].reshape(160,160,1)
#                 arr_img = list(np.array(img)[:,:,0].reshape(160,160,1))
                arr_img = list(np.array(img))

                im.append(arr_img)

                label_list = np.array(label_img.iloc[:, [2]])

                if i + '.mp4' in label_list:
                    label.append(1)  # Deepfake
                else:
                    label.append(0)
            except:
                pass
    return im, label


X = []
y = []
# all_train_dirs
for i in range(1):
    path = all_train_dirs[i]
    file_list = os.listdir(path)
    label = pd.read_csv(all_train_dirs[i] + '/metadata.csv',delimiter=',')
    img,label = read_image(path,file_list,label)
    X += img
    y += label

X = np.array(X)
y = np.array(y).reshape(-1,1)

In [ ]:
# def reduce_mem_usage(df):
#     """ iterate through all the columns of a dataframe and modify the data type
#         to reduce memory usage.        
#     """
#     start_mem = df.memory_usage().sum() / 1024**2
#     print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
#     for col in df.columns:
#         col_type = df[col].dtype
        
#         if col_type != object:
#             c_min = df[col].min()
#             c_max = df[col].max()
#             if str(col_type)[:3] == 'int':
#                 if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
#                     df[col] = df[col].astype(np.int8)
#                 elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
#                     df[col] = df[col].astype(np.int16)
#                 elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
#                     df[col] = df[col].astype(np.int32)
#                 elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
#                     df[col] = df[col].astype(np.int64)  
#             else:
#                 if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
#                     df[col] = df[col].astype(np.float16)
#                 elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
#                     df[col] = df[col].astype(np.float32)
#                 else:
#                     df[col] = df[col].astype(np.float64)
#         else:
#             df[col] = df[col].astype('category')

#     end_mem = df.memory_usage().sum() / 1024**2
#     print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
#     print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
#     return df

In [ ]:
# 데이터셋 0,1로 분류해서 인덱싱 하기 위함
c = []
d = []
e = []
f = []


for i in range(len(y)):
    if y[i] == 0:
        c.append(y[i])
        d.append(X[i])

for i in range(len(y)):
    if y[i] == 1:
        e.append(y[i])
        f.append(X[i])

X = np.array(f+d)
y = np.array(e+c).reshape(-1,1)

a=0
for i in range(len(y)):
    if y[i] == 1:
        a += 1
print(a)

print(len(X))
print(len(y))

In [ ]:
del a
del c
del d
del e
del f

In [ ]:
A = X[:88]
B = y[:88]

In [ ]:
del X
del y

In [ ]:
first_output = model.layers[-3].output
model = tf.keras.models.Model(inputs=model.input, outputs=first_output)
a = model.predict(A)
print(a.shape)

b = []
for i in range(len(a)):
    b.append(a[i].flatten())
b = np.array(b)
print(b.shape)

c = B.reshape(-1,)

In [ ]:
def divide(x, y, train_prop):
    random.seed(1234)
    tmp = np.random.permutation(np.arange(len(x)))
    X_train = x[tmp][:-(len(x)-round(train_prop * len(x)))]
    y_train = y[tmp][:-(len(x)-round(train_prop * len(x)))]
    X_test = x[tmp][-(len(x)-round(train_prop * len(x))):]
    y_test = y[tmp][-(len(x)-round(train_prop * len(x))):]
    return X_train, y_train, X_test, y_test

X_train, y_train, X_test, y_test = divide(X,y,0.7)

In [ ]:
# lightgbm 사용 시

def divide(x, y, train_prop):
    random.seed(1234)
    tmp = np.random.permutation(np.arange(len(x)))
    X_train = x[tmp][:-(len(x)-round(train_prop * len(x)))]
    y_train = y[tmp][:-(len(x)-round(train_prop * len(x)))]
    X_test = x[tmp][-(len(x)-round(train_prop * len(x))):]
    y_test = y[tmp][-(len(x)-round(train_prop * len(x))):]
    return X_train, y_train, X_test, y_test

X_train, y_train, X_val, y_val = divide(b,c,0.8)

def divide(x, y, train_prop):
    random.seed(1234)
    tmp = np.random.permutation(np.arange(len(x)))
    X_train = x[tmp][:-(len(x)-round(train_prop * len(x)))]
    y_train = y[tmp][:-(len(x)-round(train_prop * len(x)))]
    X_test = x[tmp][-(len(x)-round(train_prop * len(x))):]
    y_test = y[tmp][-(len(x)-round(train_prop * len(x))):]
    return X_train, y_train, X_test, y_test

X_train, y_train, X_test, y_test = divide(X_train,y_train,0.75)

In [ ]:
def make_feature(data_set, feature_num):
    feature_size = a.shape[1]*a.shape[2]
    data_set = data_set[:,:feature_size*feature_num]
    data_set = data_set.flatten()
    data_set = data_set.reshape(-1,400)
    return data_set
X_train = make_feature(X_train,20)
print(X_train.shape)
X_val = make_feature(X_val,20)
print(X_val.shape)
X_test = make_feature(X_test,20)
print(X_test.shape)

In [ ]:
y_train = list(y_train)
y_test = list(y_test)

def make_label(data_label,feature_num):
    d = []
    for i in range(len(data_label)):
        d.append(data_label[i]+[0]*feature_num)
    d = np.array(d)
    d = d.flatten()
    return d
y_train = make_label(y_train,20)
print(y_train.shape)
y_val = make_label(y_val,20)
print(y_val.shape)
y_test = make_label(y_test,20)
print(y_test.shape)

In [ ]:
del b
del c
del A
del B

In [ ]:
# def divide(x, y, train_prop):
#     random.seed(1234)
#     tmp = np.random.permutation(np.arange(len(x)))
#     X_train = x[tmp][:-(len(x)-round(train_prop * len(x)))]
#     y_train = y[tmp][:-(len(x)-round(train_prop * len(x)))]
#     X_test = x[tmp][-(len(x)-round(train_prop * len(x))):]
#     y_test = y[tmp][-(len(x)-round(train_prop * len(x))):]
#     return X_train, y_train, X_test, y_test

# X_train, y_train, X_test, y_test = divide(A,B,0.7)

In [ ]:
def divide(x, y, train_prop):
    random.seed(1234)
    X_train = x[:-(len(x)-round(train_prop * len(x)))]
    y_train = y[:-(len(x)-round(train_prop * len(x)))]
    X_test = x[-(len(x)-round(train_prop * len(x))):]
    y_test = y[-(len(x)-round(train_prop * len(x))):]
    return X_train, y_train, X_test, y_test

X_train, y_train, X_test, y_test = divide(k,d,0.7)

In [ ]:
rf = RandomForestClassifier(n_estimators = 50, random_state = 0)
# max_features=1,, bootstrap=False

rf.fit(X_train, y_train)

In [ ]:
y_pred = rf.predict_proba(X_test)

def assemble(test,feature_num):
    pred_test = []
    assemble_num = int(len(test)/feature_num)
    for i in range(assemble_num):
        pred_test.append(np.mean(test[feature_num*i:feature_num*(i+1),0]))
    pred_test = np.array(pred_test)
    return pred_test

y_pred_sum = assemble(y_pred,20)

def assemble_label(test,feature_num):
    pred_test = []
    assemble_num = int(len(test)/feature_num)
    for i in range(assemble_num):
        pred_test.append(np.mean(test[feature_num*i:feature_num*(i+1)]))
    pred_test = np.array(pred_test)
    return pred_test

y_test_sum = assemble_label(y_test,20)
y_test_sum = y_test_sum.astype('int64')

score = log_loss(y_test,y_pred)
print(score)

In [ ]:
import lightgbm as lgb

train_ds = lgb.Dataset(X_train, label = y_train) 
val_ds = lgb.Dataset(X_val, label = y_val) 

params = {'learning_rate': 0.01,
          'max_depth': -1,
          'boosting': 'gbdt',
          'objective': 'binary',
          'metric': 'binary',
          'is_training_metric': True,
#           'feature_fraction': 1,
          'bagging_fraction': 0.7,
          'save_binary':True,
          'scale_pos_weight':1.2,
          'seed':2020}

In [ ]:
model = lgb.train(params, train_ds, 3000, val_ds, verbose_eval=100)
# , early_stopping_rounds=100

In [ ]:
y_pred = model.predict(X_test)
score = log_loss(y_test,y_pred)
print(score)

In [ ]:
def assemble(test,feature_num):
    pred_test = []
    assemble_num = int(len(test)/feature_num)
    for i in range(assemble_num):
        pred_test.append(np.mean(test[feature_num*i:feature_num*(i+1)]))
    pred_test = np.array(pred_test)
    return pred_test

y_pred_sum = assemble(y_pred,20)

y_test_sum = assemble(y_test,20)
y_test_sum = y_test_sum.astype('int64')

In [ ]:
print(y_test_sum[:30])

In [ ]:
print(y_pred[:30])

In [ ]:
from sklearn.externals import joblib
from lightgbm import LGBMClassifier

# save model
joblib.dump(model, 'light_binary.pkl')

In [ ]:
# load model
model = joblib.load('/kaggle/input/light1/light1.pkl')

In [ ]:
y_test[1] = 1

In [ ]:
rf_probs = model.predict(X_train)
score = log_loss(y_train,rf_probs)
print(score)

In [ ]:
print(rf_probs[:500])

In [ ]:
print(y_train)